In [80]:
import shutil
import zipfile
import pandas as pd
# import xlrd  # library for reading xlsx, but can use pandas.read_excel() 
import json
import requests



def read_json_url(url):
    
    global error
    error = None
    
    try:
        r = requests.get(url, allow_redirects=True)
    except requests.exceptions.RequestException as error:    # This is the correct syntax
        print (error)
        return None

    if r.status_code == 200 and 'json' in r.headers.get('content-type',''):
        if debug: print("Good!")
        json_data = json.loads(r.text)
        return json_data
    else:
        if debug: print("Not good! Status code:" + str(r.status_code) + "    Content-type: " + r.headers.get('content-type',''))
        return None
    return


 
def load_zip_to_df(full_dst):
    zfile = zipfile.ZipFile(full_dst)
    zfile.extractall()
    
    global df

    # Read into Pandas DF
    for file_name in zfile.namelist():
        print("Loading: "+file_name)
        df = pd.read_excel(file_name)
        df.as_matrix()
        print("Records loaded: "+str(len(df)))
        break
        
    return





def count_url_items(json_url, max_depth=1):

    if debug: print("\n\n\n--------------------------")
    if debug: print("index  = "+str(index))
    if debug: print("json_url  = "+str(json_url))
    if debug: print("max_depth = "+str(max_depth))
    
    if max_depth == 0:
        if debug: print("Exiting")
        return 0
    
    max_depth -= 1
    
    json_data = read_json_url(json_url)
    if json_data == None:
        if debug: print("--- Nonetype!")
        return 0
    
    if debug: print("type(json_data) = "+str(type(json_data)))
    
    if isinstance(json_data, dict):
        for dict_key, dict_value in json_data.items():
            if isinstance(dict_value, list):
                count = len(dict_value)
                if debug: print("# of items for "+ dict_key+" : " + str(count))
                df.loc[index, dict_key] = count
                if max_depth >= 1:
                    for list_key, list_value in enumerate(dict_value):
                        count = count_url_items(list_value,max_depth)
                        #if debug: print(list_value)
            else:
                count = 1
                if debug: print("# of items for "+ dict_key+" : 1")
                df.loc[index, dict_key] = count
                if max_depth >= 1:
                    count = count_url_items(dict_value,max_depth)
                    return count
                #if debug: print(dict_value)          
    elif isinstance(json_data, list):
        count = len(json_data)
        if debug: print("# of items for : " + str(count))

    else:
        if debug: print("Error: json is not dictionary")
        return 0
    return
        


# This is a custom alternative to use of parse.urlparse(url)
# because sometimes it doesn't work well
# Simple custom rules for checking validity
def validate_url_custom(url):
    url = url.strip()
    returned_url = ''
    if url.find(' ')==-1 and url.find('.')>0 and url.find('.')<len(url)-1: 
        if not url.lower().find('http')==0:
            # Protocol not specified, but no spaces and with periods.  Try adding http://"
            url = 'http://' + url
    else:
        # Bad! Spaces or no periods"
        url = ''
        
    return url
        

def save_to_excel():    
    writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
    df.to_excel(writer,'Sheet1')
    writer.save()
    return


from IPython.display import display, clear_output
import sys


def print_same_line(print_str):
    
    '''
    #=== stdout method ===
    sys.stdout.write('\r'+print_str)
    sys.stdout.flush()
    ''' 
    
    #=== iPython method ===
    clear_output(wait=True)
    print(print_str+"\n")
    sys.stdout.flush()
    
    return
    
    
def main(start_row=0 , end_row=9):
    global df
    global index
    global error
    # Test
    src = '../DDOD-HealthData.gov/snapshots/'
    src += 'machine-readable-url-puf_2016-02-11.zip'
    dst = '.'
    print(src)
    print(dst)
    full_dst = shutil.copy2(src, dst)  # copy2 preserves metadata
    load_zip_to_df(full_dst)
    #url_dict = pd.DataFrame.to_dict(df,'records')

    print("Reading")
    

    # Loop through all rows
    #=====  Dataframe version ======
    for index, row in df.loc[start_row:end_row].iterrows():

        print_same_line("Reading row:  "+str(index)+"\n")
        
        #print(index)
        #print(row)
        url_submit = row['URL Submitted']
        url_clean = validate_url_custom(url_submit)
        if url_clean:
            row_status = count_url_items(url_clean,1)
            df.loc[index, 'Status'] = 'Read' if (error == None) else error  # Can't assign dict
        else:
            df.loc[index, 'Status'] = "Missing"
        if index >= end_row: break

    save_to_excel()
    return


In [69]:
from IPython.html.widgets import FloatProgress
from IPython.display import display
from time import sleep
f = FloatProgress(min=0, max=100)
display(f)
for i in range(100):
   sleep(0.1)
   f.value = i

In [99]:
import sys
import time

def print_same_line(print_str):
    sys.stdout.write('\r'+print_str)
    sys.stdout.flush()
'''
sys.stdout.write('some data')
sys.stdout.flush()
print("stuff")
restart_line()
sys.stdout.write('other different data')
sys.stdout.flush()
print("stuff2")

'''
print_same_line("abc")
print("\nbad stuff")
time.sleep(2) # wait 2 seconds...
print_same_line("xyz")

abc
bad stuff
xyz

In [79]:
for i in range(10):
    time.sleep(0.25)


9


In [64]:
debug = False
main(90,500)


../DDOD-HealthData.gov/snapshots/machine-readable-url-puf_2016-02-11.zip
.
Loading: Machine_Readable_PUF_2015-12-21.xlsx
Records loaded: 450
Reading
Reading row90
Reading row91
Reading row92
[SSL: UNSUPPORTED_PROTOCOL] unsupported protocol (_ssl.c:645)
Reading row93
Reading row94
Reading row95
Reading row96
Reading row97
Reading row98
Reading row99
Reading row100
Reading row101
Reading row102
Reading row103
Reading row104
Reading row105
Reading row106
Reading row107
Reading row108
Reading row109
Reading row110
Reading row111
Reading row112
Reading row113
Reading row114
Reading row115
Reading row116
HTTPSConnectionPool(host='siho.org', port=443): Max retries exceeded with url: /marketplace/cms-data-index.json (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x1067302b0>: Failed to establish a new connection: [Errno 60] Operation timed out',))
Reading row117
Reading row118
Reading row119
Reading row120
Reading row121
Reading row122
Re

In [43]:
#r = requests.get(url, allow_redirects=True)
#df[91]
#df.loc[[90:92]]
df.loc[90:93,'URL Submitted']
print(df.loc[92,'URL Submitted']) 

https://esbgatewaypub.medica.com:443/rest/QHP/cms-data-index.json?HIOSID=93078&fmt=json


In [52]:
type(df)
start = 5
end = 10
type(df.loc[start:end])
for index, row in df.loc[start:end].iterrows():
    print("Reading row"+str(index))


Reading row5
Reading row6
Reading row7
Reading row8
Reading row9
Reading row10


In [48]:
import ssl
url='https://esbgatewaypub.medica.com:443/rest/QHP/cms-data-index.json?HIOSID=93078&fmt=json'
try:
    r = requests.get(url, allow_redirects=True)
except requests.exceptions.RequestException as e:    # This is the correct syntax
    print (e)
#    sys.exit(1)

[SSL: UNSUPPORTED_PROTOCOL] unsupported protocol (_ssl.c:645)


In [10]:
df.head(5)

,Marketplace Category,State,Issuer ID,Issuer Name (Legal Name),URL Submitted,Tech POC Email,Status,formulary_urls,provider_urls,plan_urls
0,FFM,AL,17540,Guardian Life Insurance Company of America,NOT SUBMITTED,NOT SUBMITTED,Missing,NaN,NaN,NaN
1,FFM,AK,38344,Premera Blue Cross Blue Shield of Alaska,https://fm.formularynavigator.com/jsonFiles/pu...,jim.tedford@premera.com,None,1,1,1
2,FFM,AK,73836,"Moda Health Plan, Inc.",NOT SUBMITTED,NOT SUBMITTED,Missing,NaN,NaN,NaN
3,FFM,AL,44580,Humana Insurance Company,https://api.humana.com/v1/cms/index.json,pboulet@humana.com,None,99,1635,2
4,FFM,AL,46944,Blue Cross and Blue Shield of Alabama,https://www.bcbsal.org/cms/cms-data-index.json,HIMMachineReadableQuestions@bcbsal.org,None,1,3,1


In [296]:
def analyze_url(url):
    r = requests.get(url, allow_redirects=True)
    #r = requests.head(url, allow_redirects=True)
    print("Status code for "+url+": "+str(r.status_code))
    r_save = {}
    r_save['status_code'] = r.status_code
    r_save['history']     = r.history
    r_save['url']         = r.url
    r_save['_diff_url']   = (r.url == url)
    r_save['headers']     = r.headers
    r_save['elapsed']     = r.elapsed
    r_save['total_secs']  = r.elapsed.total_seconds()

    return r_save

#Test
analyze_url("http://ddod.us")

Status code for http://ddod.us: 200


{'_diff_url': False,
 'elapsed': datetime.timedelta(0, 0, 159788),
 'headers': {'X-Powered-By': 'PHP/5.5.30', 'Connection': 'Keep-Alive', 'X-Frame-Options': 'SAMEORIGIN', 'X-Mod-Pagespeed': '1.9.32.3-4523', 'Date': 'Tue, 16 Feb 2016 23:52:16 GMT', 'Content-Encoding': 'gzip', 'Content-Length': '10225', 'Content-language': 'en', 'Server': 'Apache', 'Vary': 'Accept-Encoding,Cookie', 'Cache-Control': 'private, must-revalidate, max-age=0, max-age=0, no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-UA-Compatible': 'IE=Edge', 'Keep-Alive': 'timeout=2, max=100', 'X-Content-Type-Options': 'nosniff', 'Content-Type': 'text/html; charset=UTF-8'},
 'history': [<Response [301]>],
 'status_code': 200,
 'total_secs': 0.159788,
 'url': 'http://ddod.healthdata.gov/wiki/Main_Page'}

In [293]:
r = requests.get("http://ddod.us", allow_redirects=True)
#r = requests.head(url, allow_redirects=True)
print("Status code for "+url+": "+str(r.status_code))
print(r.history)
print(r.url)

r_save = {}
r_save['status_code'] = r.status_code
r_save['history']     = r.history
r_save['url']         = r.url
r_save['_diff_url']   = (r.url == url)
r_save['headers']     = r.headers
r_save['elapsed']     = r.elapsed
r_save['total_secs']  = r.elapsed.total_seconds()


Status code for NOT DEFINED: 200
[<Response [301]>]
http://ddod.healthdata.gov/wiki/Main_Page


In [294]:
r_save

{'diff_url': False,
 'elapsed': datetime.timedelta(0, 0, 181345),
 'headers': {'X-Powered-By': 'PHP/5.5.30', 'Connection': 'Keep-Alive', 'X-Frame-Options': 'SAMEORIGIN', 'X-Mod-Pagespeed': '1.9.32.3-4523', 'Date': 'Tue, 16 Feb 2016 23:50:12 GMT', 'Content-Encoding': 'gzip', 'Content-Length': '10233', 'Content-language': 'en', 'Server': 'Apache', 'Vary': 'Accept-Encoding,Cookie', 'Cache-Control': 'private, must-revalidate, max-age=0, max-age=0, no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-UA-Compatible': 'IE=Edge', 'Keep-Alive': 'timeout=2, max=100', 'X-Content-Type-Options': 'nosniff', 'Content-Type': 'text/html; charset=UTF-8'},
 'history': [<Response [301]>],
 'status_code': 200,
 'total_secs': 0.181345,
 'url': 'http://ddod.healthdata.gov/wiki/Main_Page'}

In [285]:




r.headers['Content-Type']

'''
{'X-Powered-By': 'PHP/5.5.30', 'Connection': 'Keep-Alive', 'X-Frame-Options': 'SAMEORIGIN', 
'X-Mod-Pagespeed': '1.9.32.3-4523', 'Date': 'Tue, 16 Feb 2016 23:35:40 GMT', 
'Content-Encoding': 'gzip', 'Content-Length': '10262', 'Content-language': 'en', 
'Server': 'Apache', 'Vary': 'Accept-Encoding,Cookie', 'Cache-Control': 'private, must-revalidate, 
max-age=0, max-age=0, no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-UA-Compatible': 'IE=Edge', 
'Keep-Alive': 'timeout=2, max=100', 'X-Content-Type-Options': 'nosniff', 'Content-Type': 'text/html; charset=UTF-8'}
'''


{'X-Powered-By': 'PHP/5.5.30', 'Connection': 'Keep-Alive', 'X-Frame-Options': 'SAMEORIGIN', 'X-Mod-Pagespeed': '1.9.32.3-4523', 'Date': 'Tue, 16 Feb 2016 23:35:40 GMT', 'Content-Encoding': 'gzip', 'Content-Length': '10262', 'Content-language': 'en', 'Server': 'Apache', 'Vary': 'Accept-Encoding,Cookie', 'Cache-Control': 'private, must-revalidate, max-age=0, max-age=0, no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'X-UA-Compatible': 'IE=Edge', 'Keep-Alive': 'timeout=2, max=100', 'X-Content-Type-Options': 'nosniff', 'Content-Type': 'text/html; charset=UTF-8'}

In [288]:
r.elapsed
# datetime.timedelta(0, 0, 164833)
r.elapsed.total_seconds()  # 0.164833


'UTF-8'

In [298]:
df['Response']={}

ValueError: Length of values does not match length of index

In [326]:
# Display results
for index,row in enumerate(url_dict):
    print(row)
    print("\n------\n")
    if index == 5: break
 

{'Issuer Name (Legal Name)': 'Guardian Life Insurance Company of America', 'URL Submitted': 'NOT SUBMITTED', 'Status': 'Missing', 'Tech POC Email': 'NOT SUBMITTED', 'State': 'AL', 'Issuer ID': 17540, 'Marketplace Category': 'FFM'}

------

{'Issuer Name (Legal Name)': 'Premera Blue Cross Blue Shield of Alaska', 'URL Submitted': 'https://fm.formularynavigator.com/jsonFiles/publish/11/47/cms-data-index.json', 'Status': '{\'elapsed\': datetime.timedelta(0, 0, 964503), \'_diff_url\': True, \'headers\': {\'Server\': \'Microsoft-IIS/8.0\', \'Date\': \'Wed, 17 Feb 2016 00:21:10 GMT\', \'X-Powered-By\': \'ASP.NET\', \'Set-Cookie\': \'BIGipServerProduction-Farm-Pool-FormularyNavigator=1734803978.20480.0000; path=/\', \'Last-Modified\': \'Tue, 16 Feb 2016 23:53:50 GMT\', \'Content-Type\': \'application/json\', \'ETag\': \'"e7a2a5481569d11:0"\', \'Content-Length\': \'313\', \'Accept-Ranges\': \'bytes\'}, \'status_code\': 200, \'url\': \'https://fm.formularynavigator.com/jsonFiles/publish/11/47/cm

In [325]:
# Loop through all rows
#=====  Array of dictionary version ======

for index,row in enumerate(url_dict):
    url_submit = row['URL Submitted']
    url_clean = validate_url_custom(url_submit)
    if url_clean:
        row['Status'] = analyze_url(url_clean)
    else:
        row['Status'] = "Missing"
    if index == 5: break
   

Status code for https://fm.formularynavigator.com/jsonFiles/publish/11/47/cms-data-index.json: 200
Status code for https://api.humana.com/v1/cms/index.json: 200
Status code for https://www.bcbsal.org/cms/cms-data-index.json: 200


# Example of results #

Another
---
**Status code for https://fm.formularynavigator.com/jsonFiles/publish/11/47/cms-data-index.json: 200**

    <s>Status Code:500 Internal Server Error
    Content-Type:text/html</s>
    
    ```javascript
    {
      "provider_urls": [
        "http://fm.formularynavigator.com/jsonFiles/publish/11/47/providers.json"
      ],
                  ==> "npi":
      "formulary_urls": [
                  ==> "rxnorm_id":
        "http://fm.formularynavigator.com/jsonFiles/publish/11/47/drugs.json"
      ],
      "plan_urls": [
        "http://fm.formularynavigator.com/jsonFiles/publish/11/47/plans.json"
                  ==> "plan_id":  
      ]
    }
    ```

**Status code for https://api.humana.com/v1/cms/index.json: 200**

    Status Code:200 OK
    Content-Type:application/json; charset=utf-8
    
    
    #!/usr/bin/javascript
    {"provider_urls":["https://api.humana.com/v1/cms/pharma_providers-0.json",
                     ,"https://api.humana.com/v1/cms/pharma_providers-558.json"
                     ],
     "formulary_urls":["https://api.humana.com/v1/cms/drugs-0.json",
                      ,"https://api.humana.com/v1/cms/drugs-96.json"]
    ,"plan_urls":["https://api.humana.com/v1/cms/plans-0.json",
                  "https://api.humana.com/v1/cms/plansDental.json"]}
    
    

**Status code for https://www.bcbsal.org/cms/cms-data-index.json: 200**

    #!/usr/bin/json
    {
        "provider_urls": [
          "https://www.bcbsal.org/cms/data/providers_platinum_gold.json",
          "https://www.bcbsal.org/cms/data/providers_silver_bronze.json",
          "https://www.bcbsal.org/cms/data/providers_bcbsal.json"
        ],
        "formulary_urls": [
          "https://www.bcbsal.org/cms/data/drugs.json"
        ],
        "plan_urls": [
          "https://www.bcbsal.org/cms/data/plans.json"
        ]
    }

Status code for https://fm.formularynavigator.com/jsonFiles/publish/11/47/cms-data-index.json: 200
Status code for https://api.humana.com/v1/cms/index.json: 200
Status code for https://www.bcbsal.org/cms/cms-data-index.json: 200


In [ ]:
'''
Iterate over these collections of URLs:
    "provider_urls": [
    "formulary_urls": [
    "plan_urls": [

Count # of elements in each:
          ==> "npi":
          ==> "rxnorm_id":
          ==> "plan_id":  
          
          
FIXME: But still need to apply schema validation          
'''



In [ ]:
def analyze_count_url(url):
    r = requests.get(url, allow_redirects=True)
    #r = requests.head(url, allow_redirects=True)
    print("Status code for "+url+": "+str(r.status_code))
    r_save = {}
    r_save['status_code'] = r.status_code
    r_save['history']     = r.history
    r_save['url']         = r.url
    r_save['_diff_url']   = (r.url == url)
    r_save['headers']     = r.headers
    r_save['elapsed']     = r.elapsed
    r_save['total_secs']  = r.elapsed.total_seconds()

    return r_save

#Test
analyze_count_url("http://ddod.us")
url="https://fm.formularynavigator.com/jsonFiles/publish/11/47/cms-data-index.json"
url="https://api.humana.com/v1/cms/index.json"
url="https://www.bcbsal.org/cms/cms-data-index.json"
"""
"""

In [ ]:
import requests

url = 'https://api.humana.com/v1/cms/index.json'
url = 'http://www.healthdata.gov'
url = 'http://gray.ddod.us'
url = 'NOT SUBMITTED'
#==== URLs for testing resonse codes ====
url = 'http://httpbin.org/status/404'
url = 'http://httpbin.org/status/200'
url = 'http://httpbin.org/status/301'
url = 'healthdata.gov'

r = requests.get(url, allow_redirects=True)
#r = requests.head(url, allow_redirects=True)
print("Status code for "+url+": "+str(r.status_code))
print(r.history)
print(r.url)

In [4]:
   
    
    
url = 'https://api.humana.com/v1/cms/index.json'
url = "https://fm.formularynavigator.com/jsonFiles/publish/11/47/cms-data-index.json"
url = "https://api.humana.com/v1/cms/index.json"  # Big index.json
url = 'http://healthdata.gov'
data_json = read_json_url(url)

#response = requests.get(...)
#if json_data 
'''
r_save = {}
r_save['status_code'] = r.status_code
r_save['history']     = r.history
r_save['url']         = r.url
r_save['_diff_url']   = (r.url == url)
r_save['headers']     = r.headers
r_save['elapsed']     = r.elapsed
r_save['total_secs']  = r.elapsed.total_seconds()
'''

Not good! Status code:200    Content-type: text/html; charset=utf-8


"\nr_save = {}\nr_save['status_code'] = r.status_code\nr_save['history']     = r.history\nr_save['url']         = r.url\nr_save['_diff_url']   = (r.url == url)\nr_save['headers']     = r.headers\nr_save['elapsed']     = r.elapsed\nr_save['total_secs']  = r.elapsed.total_seconds()\n"

In [30]:
        
        
# Test
url = 'https://api.humana.com/v1/cms/index.json'
url = "https://fm.formularynavigator.com/jsonFiles/publish/11/47/cms-data-index.json"
#url = "https://api.humana.com/v1/cms/index.json"  # Big index.json
#url = 'http://healthdata.gov'
count_url_items(url, 2)




--------------------------
json_url  = https://fm.formularynavigator.com/jsonFiles/publish/11/47/cms-data-index.json
max_depth = 2
Good!
type(json_data) = <class 'dict'>
# of items for provider_urls : 1



--------------------------
json_url  = http://fm.formularynavigator.com/jsonFiles/publish/11/47/providers.json
max_depth = 1
Good!
type(json_data) = <class 'list'>
# of items for : 3256
# of items for formulary_urls : 1



--------------------------
json_url  = http://fm.formularynavigator.com/jsonFiles/publish/11/47/drugs.json
max_depth = 1
Good!
type(json_data) = <class 'list'>
# of items for : 4190
# of items for plan_urls : 1



--------------------------
json_url  = http://fm.formularynavigator.com/jsonFiles/publish/11/47/plans.json
max_depth = 1
Good!
type(json_data) = <class 'list'>
# of items for : 9


In [20]:
if 2 >= 1: print(0)

0


In [ ]:
Good!
# of items for provider_urls : 1799
# of items for formulary_urls : 97
# of items for plan_urls : 2

In [9]:
def process_url(url):
    return

In [13]:
x = 1
x -= 1
print(x)

0


In [243]:
from urllib import parse

url = 'http://www.healthdata.gov'
url = 'http://gray.ddod.us'
url = 'https://api.humana.com/v1/cms/index.json'
url = 'gray.ddod.us'  # Unfortunetly this shows up in "path" but is still usable
url = 'NOT SUBMITTED'


def validate_url(url):

    url = url.strip()
    
    # Add protocol manually if needed
    if not (url.lower().find('http') == 0):
        url = 'http://' + url


        
    if p.netloc=='':
        print("Bad URL!")
        #try_url = validate_url('http://'+url)
        #if try_url=='': try_url = validate_url('http://'+url)
        #-----------------------------
        # But is it possible that protocol was accidentally left out?
        # NEXT: check p.path for no spaces and periods
        #       Then try with http:// and https://
        #-----------------------------
        return ''

    
    print(p)

    
    return url


# Test
url = 'rand.omst.uff'
validate_url(url)
url = 'NOT SUBMITTED'
validate_url(url)
print(url)

ParseResult(scheme='http', netloc='rand.omst.uff', path='', params='', query='', fragment='')
ParseResult(scheme='http', netloc='NOT SUBMITTED', path='', params='', query='', fragment='')
NOT SUBMITTED


In [271]:

        
# Test
url = 'https://healthdata.gov'
url2 = validate_url_custom(url)
print("url: " + url2)
url = 'NOT DEFINED'
url3 = validate_url_custom(url)
print("url: " + url3)


url: https://healthdata.gov
url: 


In [96]:
#url.find('.')  #19
len(url)  #20

20

In [57]:
print(p.scheme)
print(p.netloc)

http
google.com


In [39]:
print(response_head.status_code)
print(response_head.history)

200
[]


In [35]:
type(response_head.status_code)

int

In [30]:
df[['State', 'Marketplace Category', 'URL Submitted']].\
groupby(['State', 'Marketplace Category']).agg(['count'])

,,URL Submitted
,,count
State,Marketplace Category,
AK,FFM,4
AL,FFM,8
AR,SPM,8
AZ,FFM,20
DE,SPM,7
FL,FFM,25
GA,FFM,20
IA,SPM,12


In [31]:
import requests


url_page = 'https://www.cms.gov/cciio/resources/data-resources/marketplace-puf.html'
url_file = 'http://download.cms.gov/marketplace-puf/2016/machine-readable-url-puf.zip'

response = requests.head(url_page)
#print(response.headers)
'''
{'X-Permitted-Cross-Domain-Policies': 'master-only', 'X-Frame-Options': 'SAMEORIGIN, SAMEORIGIN', 
'Connection': 'keep-alive', 'Date': 'Fri, 12 Feb 2016 15:30:03 GMT', 'Content-Length': '20', 
'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-XSS-Protection': '1; 
mode=block', 'access-control-allow-origin': '*', 'Strict-Transport-Security': 
'max-age=31536000', 'X-N': 'S', 'Content-Type': 'text/html'}
----------------------
important items:
'''



response = requests.head(url_file)
#print(response.headers)
'''
{'Server': 'Apache', 'Date': 'Fri, 12 Feb 2016 15:28:40 GMT', 'Accept-Ranges': 'bytes', 
'Last-Modified': 'Tue, 22 Dec 2015 22:02:55 GMT', 'Connection': 'keep-alive', '
ETag': '"667dbf1227e0a2ace7c89f4dc143c647:1450821775"', 'Content-Length': '28623', 
'Content-Type': 'application/zip'} 
----------------------
important items:
 'Content-Type': 'application/zip'
 'Content-Length': '28623'
 'Last-Modified': 'Tue, 22 Dec 2015 22:02:55 GMT'
 'ETag': '"667dbf1227e0a2ace7c89f4dc143c647:1450821775"   # MD5 hash of content
'''

# Simulate saving prior response
file_changed = False
for key, last_value in last_response.items():
    if not response.headers[key] == last_value:
        file_changed = True
        break  # No need to check any more 

print("---\nfile_changed: "+str(file_changed))

last_response = {'Last-Modified': response.headers['Last-Modified'],
                 'ETag' : response.headers['ETag'],
                 'Content-Length': response.headers['Content-Length']
                }
print(last_response)
'''
last_response = {'ETag': '"667dbf1227e0a2ace7c89f4dc143c647:1450821775"', 'Content-Length': '28623', 'Last-Modified': 'Tue, 22 Dec 2015 22:02:55 GMT'}
'''

NameError: name 'last_response' is not defined

In [117]:
last_response = {'ETag': '"a667dbf1227e0a2ace7c89f4d143c647:1450821775"', 'Content-Length': '28623', 'Last-Modified': 'Tue, 22 Dec 2015 22:02:55 GMT'}


In [130]:
# FIXME: last_response should come from DB
def check_file_changed(url_file,last_response):
    import requests
    
    # Check valid URL
    # return -1  # Indicates error

    response = requests.head(url_file)

    # Simulate saving prior response
    # For now last_response is global
    file_changed = False
    for key, last_value in last_response.items():
        if not response.headers[key] == last_value:
            file_changed = True
            break  # No need to check any more 

    print("---\nfile_changed: "+str(file_changed))

    #Save prior response
    last_response = {'Last-Modified': response.headers['Last-Modified'],
                     'ETag' : response.headers['ETag'],
                     'Content-Length': response.headers['Content-Length']
                    }
    
    if file_changed:
        return url_file  # Indicates changed
    else:
        return False
    
    
    
    
    
#Test
last_response = {'ETag': '"667dbf1227e0a2ace7c89f4dc143c647:1450821775"', 'Content-Length': '28623', 'Last-Modified': 'Tue, 22 Dec 2015 22:02:55 GMT'}
url_file = 'http://download.cms.gov/marketplace-puf/2016/machine-readable-url-puf.zip'
print( check_file_changed(url_file,last_response))

---
file_changed: False
False


In [175]:
from pymongo import MongoClient
from bson import ObjectId
import datetime


def check_file_changed(url_file,crawl_log):
    # Save last file loaded
    if not crawl_log.find_one({'url':url_file,'header': last_response}): 
        response_doc = {
            'url':url_file,
            'date': datetime.datetime.utcnow(),
            'header': last_response    
        }
        print("---\nInserting: "+str(document)) 
        document_id = collection.insert_one(response_doc).inserted_id
        print(document_id)
    else:
        print('Already exists')
    print(collection.count())

        
    
    
    
        for key, last_value in last_response.items():
        if not response.headers[key] == last_value:
            file_changed = True
            break  # No need to check any more 

    print("---\nfile_changed: "+str(file_changed))

    #Save prior response
    last_response = {'Last-Modified': response.headers['Last-Modified'],
                     'ETag' : response.headers['ETag'],
                     'Content-Length': response.headers['Content-Length']
                    }
    

    
    


def main():
    client = MongoClient()
    db = client.crawler  # Equiv to relational database
    crawl_log = db.crawl_log  # Equiv to table
    
    check_file_changed(url_file,crawl_log)



Already exists
1


In [194]:
# Get last updated
collection.find({'url':url_file}).sort([("id", -1)]).limit(1)


In [176]:
collection.find_one({'url':url_file,'header': last_response})
#collection.remove({'_id': ObjectId('56be05303650b51ffb96d6d4')})
#                   db.inventory.remove( { type : "food" }, 1 )

{'_id': ObjectId('56be057d3650b51ffb96d6d5'),
 'date': datetime.datetime(2016, 2, 12, 16, 17, 1, 339000),
 'header': {'Content-Length': '28623',
  'ETag': '"667dbf1227e0a2ace7c89f4dc143c647:1450821775"',
  'Last-Modified': 'Tue, 22 Dec 2015 22:02:55 GMT'},
 'url': 'http://download.cms.gov/marketplace-puf/2016/machine-readable-url-puf.zip'}

In [174]:
collection.find_one({"_id":ObjectId('56be05303650b51ffb96d6d4')})
print(collection.count())

1


In [169]:
collection.remove( {"_id": ObjectId("4d512b45cc9374271b02ec4f")});

/Users/dportnoy/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  if __name__ == '__main__':
